In [1]:
!pip install tensorflow_decision_forests
!pip install wurlitzer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 25.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    F

In [2]:
import pandas as pd
import numpy as np
import tensorflow_decision_forests as tfdf
import tensorflow as tf

In [3]:
dublin_port = pd.read_csv('dublin_port.csv')
dublin_port.head()

,Unnamed: 0,datetime,rain,temp,wetb,dewpt,vappr,rhum,msl,no2,so2,pm10,pm2.5
0,0,2022-01-01 00:00:00,0.0,13.1,11.4,9.8,12.1,80.0,1007.3,22.00,2.12,7.31,4.08
1,1,2022-01-01 01:00:00,0.0,13.9,11.2,8.5,11.1,70.0,1005.8,17.49,2.05,20.61,12.70
2,2,2022-01-01 02:00:00,0.0,14.0,11.0,8.1,10.8,67.0,1005.0,13.28,2.29,30.00,16.92
3,3,2022-01-01 03:00:00,0.0,14.6,11.4,8.4,11.0,66.0,1003.9,18.56,2.79,31.65,17.93
4,4,2022-01-01 04:00:00,0.0,14.9,12.1,9.5,11.9,70.0,1002.7,13.83,2.07,27.55,15.19


In [4]:
dublin_port = dublin_port.iloc[:, ~dublin_port.columns.isin(['Unnamed: 0'])]

total_samples = len(dublin_port)
train_samples = int(0.7 * total_samples)
val_samples = int(0.15 * total_samples)

train_data = dublin_port[:train_samples]
val_data = dublin_port[train_samples:train_samples + val_samples]
test_data = dublin_port[train_samples + val_samples:]

print("Training:", len(train_data))
print("Validation:", len(val_data))
print("Testing:", len(test_data))

Training: 6115
Validation: 1310
Testing: 1312


In [5]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label="pm2.5", task=tfdf.keras.Task.REGRESSION)
val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(val_data, label="pm2.5", task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label="pm2.5", task=tfdf.keras.Task.REGRESSION)

In [6]:
model1 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

Use /tmp/tmp_s8u7b46 as temporary training directory


In [7]:
model1.fit(train_ds)

Reading training dataset...
Training dataset read in 0:00:04.671230. Found 6115 examples.
Training model...
Model trained in 0:00:10.491797
Compiling model...


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [8]:
model1.compile(metrics=["mse", "mae"])
model1_val_result = model1.evaluate(val_ds, return_dict=True)

2/2 [==============================] - 1s 32ms/step - loss: 0.0000e+00 - mse: 4.8884 - mae: 1.5457


In [9]:
model1_val_result

{'loss': 0.0, 'mse': 4.888377666473389, 'mae': 1.5456970930099487}

In [15]:
tuner = tfdf.tuner.RandomSearch(num_trials=50)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])
local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

In [16]:
tuned_model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, tuner=tuner)
tuned_model.fit(train_ds, verbose=2)

Use /tmp/tmp4fcwu4qv as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'datetime': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'rain': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'temp': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'wetb': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'dewpt': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'vappr': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'rhum': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'msl': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'no2': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'so2': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'pm10': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>}
Label: Tensor("data_11:0", shape=(None,), dtype=float64)
Weights: None
Normalized tensor features:
 {'datetime': SemanticTensor(semantic=<Semantic.CATEGORICAL: 2>, tensor=<tf.Tensor 'data:0' shape=(None,) dtype=string>), 

[INFO 23-07-11 20:47:21.1245 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-07-11 20:47:21.1245 UTC kernel.cc:774] Collect training examples
[INFO 23-07-11 20:47:21.1245 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-07-11 20:47:21.1246 UTC kernel.cc:393] Number of batches: 7
[INFO 23-07-11 20:47:21.1246 UTC kernel.cc:394] Number of examples: 6115
[INFO 23-07-11 20:47:21.1274 UTC data_spec_inference.cc:305] 6115 item(s) have been pruned (i.e. they are considered out of dictionary) for the column datetime (0 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 23-07-11 20:47:21.1287 UTC kernel.cc:794] Training dataset:
Number of records: 6115
Number of columns: 12

Numb

Model trained in 0:02:29.072584
Compiling model...
Model compiled.


In [18]:
tuning_logs = tuned_model.make_inspector().tuning_logs()
tuning_logs

,score,evaluation_time,best,min_examples,categorical_algorithm,growing_strategy,max_num_nodes,num_candidate_attributes_ratio,max_depth
0,-2.096330,4.183021,False,2,RANDOM,BEST_FIRST_GLOBAL,32.0,0.5,NaN
1,-2.124673,8.401032,False,7,RANDOM,LOCAL,NaN,0.5,6.0
2,-1.903441,10.810021,False,7,CART,BEST_FIRST_GLOBAL,256.0,1.0,NaN
3,-2.123331,14.809528,False,10,CART,LOCAL,NaN,0.5,6.0
4,-2.403478,16.695020,False,7,CART,LOCAL,NaN,0.5,3.0
5,-1.905756,19.664944,False,2,CART,BEST_FIRST_GLOBAL,256.0,0.9,NaN
6,-2.167794,22.662144,False,10,CART,BEST_FIRST_GLOBAL,16.0,1.0,NaN
7,-2.403478,25.584292,False,2,CART,LOCAL,NaN,0.5,3.0
8,-2.013989,28.856135,False,10,CART,BEST_FIRST_GLOBAL,64.0,0.9,NaN
9,-1.943720,34.977546,False,7,CART,BEST_FIRST_GLOBAL,128.0,0.5,NaN


In [19]:
tuned_model.compile(metrics=["mse", "mae"])
tuned_model_val_result = tuned_model.evaluate(val_ds, return_dict=True)

2/2 [==============================] - 0s 25ms/step - loss: 0.0000e+00 - mse: 4.9099 - mae: 1.5440


In [20]:
tuned_model_val_result

{'loss': 0.0, 'mse': 4.909885406494141, 'mae': 1.5439581871032715}

In [21]:
auto_tuner = tfdf.tuner.RandomSearch(num_trials=50, use_predefined_hps=True)

In [22]:
auto_tuned_model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, tuner=auto_tuner)
auto_tuned_model.fit(train_ds, verbose=2)

Use /tmp/tmp82fc7wdf as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'datetime': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'rain': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'temp': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'wetb': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'dewpt': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'vappr': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'rhum': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'msl': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'no2': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'so2': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'pm10': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>}
Label: Tensor("data_11:0", shape=(None,), dtype=float64)
Weights: None
Normalized tensor features:
 {'datetime': SemanticTensor(semantic=<Semantic.CATEGORICAL: 2>, tensor=<tf.Tensor 'data:0' shape=(None,) dtype=string>), 

Training dataset read in 0:00:00.197071. Found 6115 examples.
Training model...


[INFO 23-07-11 21:00:11.4663 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-07-11 21:00:11.4663 UTC kernel.cc:774] Collect training examples
[INFO 23-07-11 21:00:11.4663 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-07-11 21:00:11.4665 UTC kernel.cc:393] Number of batches: 7
[INFO 23-07-11 21:00:11.4665 UTC kernel.cc:394] Number of examples: 6115
[INFO 23-07-11 21:00:11.4703 UTC data_spec_inference.cc:305] 6115 item(s) have been pruned (i.e. they are considered out of dictionary) for the column datetime (0 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 23-07-11 21:00:11.4739 UTC kernel.cc:794] Training dataset:
Number of records: 6115
Number of columns: 12

Numb

Model trained in 1:00:57.746074
Compiling model...
Model compiled.


In [23]:
auto_tuned_model.compile(metrics=["mse", "mae"])
auto_tuned_model_val_result = auto_tuned_model.evaluate(val_ds, return_dict=True)

2/2 [==============================] - 0s 62ms/step - loss: 0.0000e+00 - mse: 5.1870 - mae: 1.5876


In [24]:
auto_tuned_model_val_result

{'loss': 0.0, 'mse': 5.186985015869141, 'mae': 1.5876216888427734}

In [26]:
auto_tuning_logs = auto_tuned_model.make_inspector().tuning_logs()
auto_tuning_logs

,score,evaluation_time,best,split_axis,sparse_oblique_projection_density_factor,sparse_oblique_normalization,sparse_oblique_weights,categorical_algorithm,winner_take_all,max_depth,min_examples
0,-1.994781,115.138856,False,SPARSE_OBLIQUE,5.0,MIN_MAX,BINARY,CART,true,12,40
1,-1.868576,216.717563,False,SPARSE_OBLIQUE,5.0,MIN_MAX,CONTINUOUS,CART,true,16,1
2,-1.882709,265.533013,False,SPARSE_OBLIQUE,4.0,MIN_MAX,CONTINUOUS,RANDOM,true,16,10
3,-1.877557,375.700683,False,SPARSE_OBLIQUE,1.0,MIN_MAX,CONTINUOUS,CART,true,20,10
4,-1.851542,441.910827,False,SPARSE_OBLIQUE,4.0,STANDARD_DEVIATION,BINARY,CART,true,25,2
5,-1.884519,500.094756,False,SPARSE_OBLIQUE,2.0,NONE,BINARY,CART,true,20,10
6,-1.890554,609.627035,False,SPARSE_OBLIQUE,5.0,MIN_MAX,BINARY,RANDOM,true,20,10
7,-1.872225,694.771593,False,SPARSE_OBLIQUE,5.0,MIN_MAX,BINARY,RANDOM,true,30,5
8,-1.988227,777.396740,False,SPARSE_OBLIQUE,1.0,NONE,CONTINUOUS,RANDOM,true,25,40
9,-1.850238,818.074962,False,SPARSE_OBLIQUE,3.0,NONE,CONTINUOUS,CART,true,30,2
